# Module to make latex tables using pandas.

### Modes of making latex table:
- Simple table using gvar
- Table with powers of 10 without gvar.
- Table with val(error) without gvar.
    
    
#### Nov 14, 2017.

## Steps involved:
- Define functions
- Read data from csv into Dataframe and organize index
- Create latex code for table
- Write latex code to a blank .tex file to check whether it formats correctly.

In [1]:
import pandas as pd
import numpy as np
import os
import subprocess as sp
import glob


In [2]:
# Uses gvar 

import gvar as gv

def f_gvarize(df,col1,col2):
    '''
    Get Dataframe column with val(err).
    Get val(err) using gvar 
    
    '''
    df1=df.copy()
    col3=[gv.gvar(j[col1],j[col2]) for i,j in df.iterrows()]
    df2=df1.drop(labels=[col1,col2],axis=1)

    df2[col1]=np.array(col3,dtype=str)

    return df2


## Explanation for frexp10

Given $ p $ we need to find $ a $ and $ b $ such that:
$$ p = a \times 10^b .$$
Taking log on both sides:
$$ \log p = \log_{10}(a) + b $$
Split $ \log p $ into interger part + decimal $$ \log p = Int[\log p] + Dec[\log p] = x + y  .$$
Therefore, $ b = x  $ and $ \log_{10}(a) = y $. (Since 'b' is defined to be an integer. We want 10^{integer}.)
  
Hence, $ a = 10^{y} $. 


For -ve p, need modification



In [3]:
## Functions for fancy latex table with powers of 10.

def frexp10(p):
    '''
    Function to express a positive floating point number as :   y=a 10^b
    log_10 p = log_10 (a) + b = x = dec + b
    for p>1, b is +ve, so integer part of x (=b) is power and 10**dec is 
    for p<0, modification required. b = b-1, dec = 1 + dec
    
    '''
    L=np.log10(p) # get log to the base 10
    M=np.modf(L) # split a float into integer part and rest. M[0] is decimal part, M[1] is integer part
    
    y,x=M[0],M[1]
    
    if p < 1.0 :
        y=1.0+y
        x=x-1
        
    a=10**y
    b=x
    fcoeff,fpow=a,b
    
    return fcoeff,fpow
    

def f_value_error(c1,c2,mode='latex',float_type='eng'):
    '''
    Function that reads two numbers : value and error.
    It finds the power of the error(c2), extracts 1st digit of error and uses that to write value(c1).
    
    Two modes for float_type:
    1. float_type:'scientific'   : gives string of the form value(error) \times 10^{exponent}.
    2. float_type:'eng' (default): gives string of the form value(error).
    Two modes:
    1. mode 'latex' : Gives string that can be used directly in latex file.
    2. mode 'normal': Gives a value(eng) or a list(scientific).
    
    Need an exception if c2 = 0.
    Two cases for 'eng' for error>1 and error <1.
    
    
    
    '''
    
    # Exception for c2=0
    if c2==0:
        if mode=='latex':
            return '$'+str(c1)+'$'
        elif mode=='normal':
            return [str(c1)]
        
    else :
        a=frexp10(c2)
        b=np.rint(c1/(10**a[1]))
        c=np.around(a[0])
        
        # For 'eng' type, two cases for error >1 and error <1.
        if float_type=='eng':
            if a[1]<0.0: # For power_of_error<0.0 ie error<1, print error as it is
                # But, values needs some careful handling.
                # If it has zeros at the end, str(float) trucates it. 
                # So, use the power of 10 to get number of significant digits and make string accordingly.
                str_temp="%df"%(int(np.abs(a[1])))
                prnt_str="%."+str_temp                 
                d=prnt_str%(b*(10**a[1]))+'('+str(int(c))+')' # amounts to "%.3f"%(val) + err
            
            elif a[1]>=0.0: # For error >1, multiply error with value and print it
                d=str(int(b*(10**a[1])))+'('+str(int(c*(10**a[1])))+')'

#     print a,b,c
    try:
        if mode=='latex':
            if float_type=='scientific':
                ans='$'+ str(int(b))+'('+str(int(c))+') \\times 10^{'+str(int(a[1]))+'}$'
            elif float_type=='eng':
                ans='$'+d+'$'
                
        elif mode=='normal' :
            if float_type=='scientific':
                ans=[str(int(b))+'('+str(int(c))+')','10^{'+str(int(a[1]))+"}"]
            elif float_type=='eng':
                ans=d
    except Exception as e:
        print e
        print "Original values",c1,c2
        print "a,b,c",a,b,c,
        raise SystemError
        
    return ans

def f_df_val_err(df,col1,col2,mode='latex',float_type='eng'):
    '''
    Code to get column with val(err) in dataframe for 2 columns
    Reads dataframe and the two column names,
    Uses 'f_value_error' function on each pair of numbers
    '''
    
    df2=df[:]    
    gg=[f_value_error(col[col1],col[col2],mode=mode,float_type=float_type) for row,col in df.iterrows()]
    df3=df2.drop(labels=[col1,col2],axis=1)
    df3[col1]=np.array(gg,dtype=str)
    return df3


In [5]:
# Testing modules and getting data in DataFrame

if __name__=="__main__":
    print "Testing modules"
    print frexp10(2.00)
    print f_value_error(2.00,0.003)

if __name__=="__main__":
    write_data_dir='test_data/'
    fname=write_data_dir+'gathered_data_df.txt'
    df5=pd.read_csv(fname,sep='\t',header=0,names=['U','L','chi','dchi'],index_col = False).reset_index().drop('index',1)
    df5['L']=df5['L'].astype(int)
    df6=f_df_val_err(df5,'chi','dchi')
    df7=f_gvarize(df5,'chi','dchi')

Testing modules
(2.0, 0.0)
$2.0(3)$


In [7]:
# Write table to a file 
if __name__=="__main__":
    write_data_dir='test_data/'
    
    ############# Get latex code #############
    # Stuff to write to a blank latex file
    # Don't forget to add the following packages to latex document: \usepackage{booktabs,longtable}
    doc_pre="\documentclass{article}\n\usepackage{longtable}\n\usepackage{booktabs}\n"
    doc_pre="\documentclass{article}\n\usepackage{longtable,booktabs}\n\\begin{document}"
    doc_end="\end{document}"
    table_pre="\n\\begin{table}[htb]\n"
    table_end="\end{table}"
    page_break="\\newpage"
    
    # Create latex code from a dataframe
    df=df6[:]
    # Rename columns to include $ $ signs.
    df.rename(columns={'chi':'$ \chi $','L':'$ L $ ','U': '$U$'},inplace=True)
    # Write to latex
    table_form=df.to_latex(index=False,
                           column_format='|c|l||r|',
                           longtable=False,
                           escape=False)
    caption='\caption{Values for $\\langle n \\rangle$ for square lattices with open boundary conditions\
, used in Fig. 8. Here $ U=0.3$ and $ L = 40,64$. }\n'
    table_form+=caption
    
    
    ######## Now write everything to file ##########
    
    %store doc_pre > latex_table/table.tex
    # Writing table 1
    %store table_pre >> latex_table/table.tex
    %store table_form >> latex_table/table.tex
    %store table_end >> latex_table/table.tex
    %store page_break >> latex_table/table.tex
    %store doc_end >> latex_table/table.tex


Writing 'doc_pre' (str) to file 'latex_table/table.tex'.
Writing 'table_pre' (str) to file 'latex_table/table.tex'.
Writing 'table_form' (unicode) to file 'latex_table/table.tex'.
Writing 'table_end' (str) to file 'latex_table/table.tex'.
Writing 'page_break' (str) to file 'latex_table/table.tex'.
Writing 'doc_end' (str) to file 'latex_table/table.tex'.


## For more details, refer to the 'gather_data_Thirring_project' notebook.
### There are more details about longtable, splitting up tables side by side, etc.